# Pivot Tables

Hemos visto cómo la abstracción `` GroupBy `` nos permite explorar las relaciones dentro de un conjunto de datos.

Una *pivot table* o tabla dinámica es una operación similar que se ve comúnmente en hojas de cálculo y otros programas que operan con datos tabulares.
La tabla dinámica toma datos simples de columnas como entrada y agrupa las entradas en una tabla bidimensional que proporciona un resumen multidimensional de los datos.

La diferencia entre tablas dinámicas y `` GroupBy `` a veces puede causar confusión; pero básicamente lo podemos entender como que las tablas dinámicas son una versión multidimensional de la agregación `` GroupBy ``.

Es decir, divide-aplica-combina, pero tanto la división como la combinación ocurren no en un índice unidimensional, sino en una cuadrícula bidimensional.

En este notebook, utilizaremos los datos que ya hemos visto de los pasajeros del Titanic para desarrollar los conceptos. Datos disponibles a través de la librería Seaborn.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [2]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Este dataset contiene una gran cantidad de información sobre cada pasajero, incluyendo el sexo, la edad, la clase, la tarifa pagada y mucho más.

## Pivot Tables a mano

Para comenzar a aprender más sobre estos datos, podríamos comenzar agrupando según el género, si ha sobrevivido o no, o alguna combinación de los mismos.

En base a lo que ya hemos aprendido, podríamos tener la tentación de aplicar una operación de `` GroupBy ``; por ejemplo, veamos la tasa de supervivencia por género:

In [3]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


Esto inmediatamente nos da una idea: en general, tres de cada cuatro mujeres a bordo sobrevivieron, mientras que solo uno de cada cinco hombres lo hizo.

Esto es útil, pero nos gustaría ir un paso más allá y estudiar la supervivencia por sexo y, por ejemplo, clase.
Usando el `` GroupBy ``, podríamos proceder usando algo como:
Agrupamos por clase y género, *seleccionamos* 'survival, * aplicamos * la agregación de la media, * combinamos * los grupos resultantes, y luego * desapilamos * el índice jerárquico para revelar la multidimensionalidad oculta. En codigo:

In [4]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Esto nos da una idea más precisa de cómo el género y la clase afectaron la supervivencia, pero el código comienza a volverse un tanto confuso.

Si bien cada paso que damos tiene sentido dadas las herramientas que hemos visto, la larga cadena de código no es particularmente fácil de leer o usar.

Este `` GroupBy `` bidimensional es lo suficientemente común como para que Pandas incluya algo que nos facilite su tratamiento, como son las `` pivot_table ``, diseñadas especialmente para este tipo de agregación multidimensional.

## Sintaxis de Pivot Table

A continuación, se recoge el equivalente a la operación anteriore mediante ``Pivot Table``, la cual se puede obtener directamente con el método de los DataFrames `` pivot_table``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Esto es mcho más legible que el enfoque de `` group by `` y produce el mismo resultado.

Como era de esperar de un crucero transatlántico de principios del siglo XX, el gradiente de supervivencia favorece tanto a las mujeres como a las clases altas.
Las mujeres de primera clase sobrevivieron con un porcentaje muy elevado, mientras que solo uno de cada diez hombres de tercera clase lo hizo.

### Pivot Tables multinivel

Al igual que en el `` GroupBy ``, la agrupación en tablas dinámicas se puede especificar con múltiples niveles y mediante varias opciones.

Por ejemplo, podríamos estar interesados en considerar la edad como una tercera dimensión. Pero meter un dato de una variable con tanta variación podría desagregarnos tanto los datos que no sacaríamos nada en concreto. Por ello, resultaría más interesante realizar cierta agrupación de edades, como hacer una división entre menores de edad (<18), adultos (>18 y <65) y jubilados (>65), lo cual lo podemos conseguir gracias a la función `` pd.cut ``:

In [19]:
age = pd.cut(titanic['age'], [0, 18, 65, int(max(titanic['age']))])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 65]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 65]  0.380435  0.073171  0.135000
       (65, 80]  0.250000  0.000000  0.000000

También podríamos aplicar la misma estrategia cuando trabajamos con las columnas. Por ejemplo, podríamos agregar información sobre la tarifa pagada usando `` pd.qcut `` para calcular automáticamente los cuantiles:

In [20]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 65]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 65]              0.0  0.102041  0.126437          0.397727   
       (65, 80]              NaN  0.000000  0.000000          0.250000   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 65]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 65]  0.030303  0.192308  
       (65, 80]       NaN       NaN

El resultado es una agregación de cuatro dimensiones con índices jerárquicos, que se muestra en una cuadrícula que demuestra la relación entre los valores.

### Opciones adicionales de Pivot Table

La llamada completa del método `` pivot_table `` de `` DataFrame `` es la siguiente:

```python
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

Ya hemos visto ejemplos de los tres primeros argumentos; por lo que ahora echaremos un vistazo rápido a los restantes.
Dos de las opciones, `` fill_value `` y `` dropna ``, tienen que ver con datos faltantes y son bastante sencillas; por lo que no mostraremos ejemplos de ellos aquí.

Por otra parte, la palabra clave ``aggfunc`` controla qué tipo de agregación se aplica, que por defecto será la media.
Al igual que con el `` GroupBy ``, la especificación de agregación puede ser un string que represente una de varias opciones comunes (por ejemplo, ``sum``, ``mean``, ``count``, ``min``, ``max``...) o una función que implemente una agregación (por ejemplo, `` np.sum ()`` , `` min ()``, `` sum ()``...).

Además, se puede especificar como un diccionario que asigne una columna a cualquiera de las opciones deseadas anteriores:

In [22]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Date cuenta que también aquí que hemos omitido la palabra clave `` values ``, ya que al especificar un mapeo para "aggfunc", este parámetro se determina automáticamente.

También podríamos calcular en base a una combinación de ejes:

In [25]:
titanic.pivot_table(index=['sex', 'embarked'], columns=['class', 'alone'],
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                                               \
class                 First                 Second                 Third   
alone                 False       True       False      True       False   
sex    embarked                                                            
female C         112.003000  120.692128  30.217500  12.895850  16.367589   
       Q          90.000000         NaN        NaN  12.350000  18.284375   
       S         102.443874   90.728571  27.621795  13.960714  23.898852   
male   C         119.409874   72.163222  32.896533  14.208325  13.103770   
       Q          90.000000         NaN        NaN  12.350000  21.815000   
       S          92.783489   32.267146  30.441667  14.213433  24.354565   

                          survived                                 
class                        First       Second       Third        
alone               True     False True   False True  False True   
sex    embarked                                                    
female C         8.673340     25.0  17.0    5.0   2.0  11.0   4.0  
       Q         7.755340      1.0   NaN    NaN   2.0   5.0  19.0  
       S         8.561113     32.0  14.0   36.0  25.0  19.0  14.0  
male   C         8.215409      9.0   8.0    1.0   1.0   5.0   5.0  
       Q         8.513648      0.0   NaN    NaN   0.0   1.0   2.0  
       S         9.861668     11.0  17.0    9.0   6.0   9.0  25.0

A veces es útil calcular los totales de cada agrupación. Esto se puede hacer mediante el parámetro `` margins ``:

In [26]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


Aquí, esto nos da automáticamente información sobre la tasa de supervivencia por género o por clase sin tener en cuenta el otro eje, gracias a los totales. Además, en el registro correspondiente a los dos ``All``, el resultado se corresponde con el total sin tener en cuenta ninguna división, es decir, nos dice que el porcentaje de supervivientes total fue del 38%.

La etiqueta del "margen" que hemos añadido se puede especificar con el parámetro `` margins_name``, que por defecto es `` All ``.

## Ejemplo: Cumpleaños

Veamos ahora un ejemplo más interesante, echemos un vistazo a los datos disponibles sobre nacimientos en los Estados Unidos, proporcionados por los Centros para el Control de Enfermedades (CDC).

Estos datos se pueden encontrar en https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv

In [28]:
births = pd.read_csv('births.csv')

Al observar los datos, vemos que es relativamente simple: contiene la cantidad de nacimientos agrupados por fecha y sexo:

In [29]:
births.head()

,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548


Podemos comenzar a comprender estos datos un poco más usando una pivot table.

Por ejemplo, agreguemos una columna de décadas y echemos un vistazo a los nacimientos de hombres y mujeres en función de la década:

In [ ]:
births['decade'] = 10 * (births['year'] // 10)
births.pivot_table('births', index='decade', columns='gender', aggfunc='sum')

gender,F,M
decade,,
1960,1753634,1846572
1970,16263075,17121550
1980,18310351,19243452
1990,19479454,20420553
2000,18229309,19106428


A simple vista, vemos que los nacimientos masculinos superan en número a los nacimientos femeninos en cada década.

## Ejercicio

Utiliza el fichero "births.csv" para responder a las siguientes preguntas:

1. Elimina aquellos registros que tengan algún registro con nulos
2. Crea un ``DataFrame`` con las columnas "year" y "gender" como índice. Ordena el ``DataFrame`` por el índice que has creado. Quédate con los nacimientos de mujeres de 1971. ¿Cuántos nacimientos hubo? Calcula el resultado utilizando el acceso por índice y no mediante el filtrado por condición.
3. Obtén como ``Series`` la suma de nacimientos por mes y género, usando "groupby". Convierte ese ``Series`` en un ``DataFrame`` con los meses como índice y el género como columnas.
4. Repite el ejercicio 3 con una pivot_table
5. Calcula el máximo de nacimientos en función del mes, para cada combinación de género y la siguiente agrupación de años: `` [1968, 1972), [1972, 1976), [1976, 1980), [1980, 1984), [1984, 1989]``. Utiliza alguna de las funciones que hemos visto en este notebook. (En el índice deberán estar tanto el género como cada una de las agrupaciones de años, y en las columnas el nº de mes).
6. Sobre el ``DataFrame`` anterior, accede a los nacimientos de varones entre los años 1972 y 1976, el mes de Febrero. Puede que necesites investigar lo que tienes y pensar un poco. Sabemos que tenemos que acceder con el índice, pero no sabemos qué debemos poner en el índice. Investiga con algún valor de los que tenemos en el índice. Pista: si llegas a un punto en el que pones algo de "Interval" y falla, sustituye ``Interval`` por ``pd.Interval`` ;)
7. Repite el ejercicio 5, pero calculando la media en lugar del máximo, y agrupando, en lugar de por años, por días, de forma que queden los primeros 10 días del mes, del 11 al 20, y del 21 al... Investiga los datos y piensa una solución para decidir hasta cuándo tomar. Encuentra la que más te convenza y úsala, o prueba varias

In [51]:
import pandas as pd

df = pd.read_csv("births.csv")
df

,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548
...,...,...,...,...,...
15542,2008,10,NaN,M,183219
15543,2008,11,NaN,F,158939
15544,2008,11,NaN,M,165468
15545,2008,12,NaN,F,173215


In [52]:
# 1. 
df = df.dropna()

In [88]:
# 2. 
df2 = df.copy().set_index(["year", "gender"])
df2 = df2.sort_index()
print(df2.loc[(1971, "F"), :])
print("Total nacimientos Mujeres 1971: " + str(df2.loc[(1971, "F"), :]['births'].sum()))

             month   day  births
year gender                     
1971 F           1   1.0    4214
     F           1   2.0    4460
     F           1   3.0    4288
     F           1   4.0    4868
     F           1   5.0    5012
...            ...   ...     ...
     F          12  28.0    5218
     F          12  29.0    4936
     F          12  30.0    5048
     F          12  31.0    4520
     F          12  99.0      18

[381 rows x 3 columns]
Total nacimientos Mujeres 1971: 1736774


In [93]:
# 3. 
ser1 = df.groupby(["month", "gender"]).apply(lambda x: x["births"].sum())
ser1.unstack()

gender,F,M
month,,
1,2810078,2949490
2,2616914,2746503
3,2859813,3008838
4,2706556,2854761
5,2813046,2972556
6,2798479,2960754
7,3026280,3194497
8,3075596,3234680
9,3029496,3177289


In [94]:
# 4. 
df.pivot_table('births', index='month', columns='gender', aggfunc='sum')

gender,F,M
month,,
1,2810078,2949490
2,2616914,2746503
3,2859813,3008838
4,2706556,2854761
5,2813046,2972556
6,2798479,2960754
7,3026280,3194497
8,3075596,3234680
9,3029496,3177289


In [126]:
# 5. 
years = pd.cut(df['year'], [1968, 1972, 1976, 1980, 1984, 1989])
df5 = df.pivot_table('births', index=['gender', years], columns='month', aggfunc='max')
df5

month                  1     2     3     4     5     6     7     8     9   \
gender year                                                                 
F      (1968, 1972]  5542  5386  5334  5222  5264  5494  5752  5656  5986   
       (1972, 1976]  4541  4583  4634  4489  4655  4551  4909  5001  5156   
       (1976, 1980]  5029  5054  5171  4976  5186  5347  5516  5613  5667   
       (1980, 1984]  5210  5518  5294  5222  5341  5514  5761  5813  6021   
       (1984, 1989]  5464  5652  5615  5611  5744  5910  6200  6102  6332   
M      (1968, 1972]  5682  5684  5972  5398  5494  5884  6208  6138  6480   
       (1972, 1976]  4879  4887  4889  4762  4899  4955  5204  5227  5343   
       (1976, 1980]  5234  5378  5385  5437  5385  5608  5945  5879  6055   
       (1980, 1984]  5498  5747  5597  5537  5672  5942  6119  6139  6250   
       (1984, 1989]  5767  5842  5888  6054  6172  6283  6527  6370  6519   

month                  10    11    12  
gender year                            
F      (1968, 1972]  5786  5610  6204  
       (1972, 1976]  4844  4633  4974  
       (1976, 1980]  5417  5229  5645  
       (1980, 1984]  5672  5461  5717  
       (1984, 1989]  5878  5826  6092  
M      (1968, 1972]  5796  5836  6244  
       (1972, 1976]  5420  4969  5210  
       (1976, 1980]  5714  5536  5958  
       (1980, 1984]  5935  5678  5988  
       (1984, 1989]  6205  6072  6343

In [123]:
# 6. 
df5.loc[('M', pd.Interval(1968, 1972, closed='right')), 2]

5684

In [129]:
# 7. 
days = pd.cut(df['day'], [0, 10, 20, 31])
df7 = df.pivot_table('births', index=['gender', days], columns='month', aggfunc='mean')
df7

month                     1            2            3            4   \
gender day                                                            
F      (0, 10]   4395.215000  4599.140000  4647.545000  4538.285000   
       (10, 20]  4604.535000  4653.630000  4612.150000  4498.780000   
       (20, 31]  4590.559091  4141.227027  4580.181818  4301.172249   
M      (0, 10]   4618.125000  4834.085000  4873.985000  4780.155000   
       (10, 20]  4831.845000  4885.675000  4859.820000  4743.535000   
       (20, 31]  4814.990909  4432.668508  4826.363636  4566.379808   

month                     5            6            7            8   \
gender day                                                            
F      (0, 10]   4500.375000  4626.660000  4798.845000  4953.310000   
       (10, 20]  4562.855000  4646.190000  4887.995000  4985.700000   
       (20, 31]  4546.381818  4536.653846  4948.559091  4943.404545   
M      (0, 10]   4758.860000  4881.590000  5073.440000  5205.900000   
       (10, 20]  4807.755000  4914.750000  5164.885000  5239.620000   
       (20, 31]  4813.850000  4813.625000  5211.722727  5205.950000   

month                     9            10           11           12  
gender day                                                           
F      (0, 10]   4915.435000  4924.130000  4668.440000  4654.290000  
       (10, 20]  5096.095000  4732.405000  4687.055000  4751.520000  
       (20, 31]  4913.287081  4656.345455  4331.885167  4557.986364  
M      (0, 10]   5175.780000  5146.280000  4901.765000  4875.515000  
       (10, 20]  5345.340000  4963.950000  4922.690000  4971.560000  
       (20, 31]  5108.409524  4898.286364  4543.495238  4777.463636

## Ejercicio 2

Lee nuestro más que conocido fichero con datos de jugadores de fútbol ("FIFA20.csv") y construye las siguientes pivot table:

1. Suma de salarios de jugadores (en millones) en base a su team_position, su preferred_foot y si es influencer o no. Elige las columnas y los índices en función de lo que ceas oportuno. Igual te sirve como consejo que puedes utilizar funciones creadas por ti en el aggfunc, que recibirán cada grupito de registros, como si de un groupby se tratara
2. Media de potential y mínimo de overall en función del año de nacimiento agrupado según su década utilizando los buckets oportunos, y el value_eur agrupado en los buckets [0, 100 000), [100 000, 1 000 000), [1 000 000, max). Para obtener el año, podríamos utilizar un apply... Pero tranquilo, que en el tema siguiente veremos algún truquillo. Para sacar los grupos de las décadas, quiero que mires los años que tienes y que saques manualmente los años para hacer los buckets (aunque si eres capaz de sacarlo por código, adelante). En cuanto a la aggfunc, piensa en qué hacíamos para agregar con los group by

In [130]:
import pandas as pd

df = pd.read_csv("FIFA20.csv")
df

,short_name,long_name,dob,club,overall,potential,value_eur,wage_eur,team_position,preferred_foot,influencer
0,L. Messi,Lionel Andrés Messi Cuccittini,1987-06-24,FC Barcelona,94,94,95500000,565000,RW,Left,True
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,1985-02-05,Juventus,93,93,58500000,405000,LW,Right,True
2,Neymar Jr,Neymar da Silva Santos Junior,1992-02-05,Paris Saint-Germain,92,92,105500000,290000,CAM,Right,True
3,J. Oblak,Jan Oblak,1993-01-07,Atlético Madrid,91,93,77500000,125000,GK,Right,False
4,E. Hazard,Eden Hazard,1991-01-07,Real Madrid,91,91,90000000,470000,LW,Right,True
...,...,...,...,...,...,...,...,...,...,...,...
18273,Shao Shuai,邵帅,1997-03-10,Beijing Renhe FC,48,56,40000,2000,RES,Right,False
18274,Xiao Mingjie,Mingjie Xiao,1997-01-01,Shanghai SIPG FC,48,56,40000,2000,SUB,Right,False
18275,Zhang Wei,张威,2000-05-16,Hebei China Fortune FC,48,56,40000,1000,SUB,Right,False
18276,Wang Haijian,汪海健,2000-08-02,Shanghai Greenland Shenhua FC,48,54,40000,1000,SUB,Right,False


In [140]:
# 1. 

def suma_millones(x):
    return sum(x)/1000000

df.pivot_table("wage_eur", index = ["team_position"], columns=["preferred_foot", "influencer"], aggfunc=suma_millones)
# df.pivot_table("wage_eur", index = ["team_position"], columns=["preferred_foot", "influencer"], aggfunc=lambda x: np.sum(x)/1000000)

preferred_foot    Left          Right       
influencer       False  True    False  True 
team_position                               
CAM              0.999  0.125   2.438  0.460
CB               0.078    NaN   1.164    NaN
CDM              0.303    NaN   2.567  0.313
CF               0.053    NaN   0.386  0.285
CM               0.109    NaN   0.839    NaN
GK               1.121  0.385   5.586  0.360
LAM              0.068    NaN   0.172    NaN
LB               5.028  0.340   0.332    NaN
LCB              3.089  0.215   4.014  0.701
LCM              1.285  0.265   3.742  0.380
LDM              0.634    NaN   1.818  0.250
LF               0.149    NaN   0.188    NaN
LM               1.281    NaN   3.337    NaN
LS               0.449    NaN   2.347    NaN
LW               0.232  0.520   2.149  0.875
LWB              0.417    NaN   0.140    NaN
RAM              0.125    NaN   0.108    NaN
RB               0.020    NaN   5.626    NaN
RCB              0.332    NaN   7.383  0.309
RCM              0.577    NaN   4.457  0.955
RDM              0.175    NaN   2.385    NaN
RES              3.721  0.011  11.708    NaN
RF               0.147    NaN   0.158    NaN
RM               0.992    NaN   3.650    NaN
RS               0.620    NaN   2.016  0.027
RW               1.472  0.588   1.325    NaN
RWB              0.008    NaN   0.577    NaN
ST               0.847    NaN   5.575  1.113
SUB             15.749  0.475  46.574  1.061

In [171]:
# 2. 

# Sacamos años:
years_dob = df['dob'].apply(lambda x: int(x[:4]))

# Sacamos programáticamente los años para los buckets (esto estaba pensado para que mirárais cuáles eran los años de las décadas y los pusiérais a mano, 
#no por código, pero os lo dejo por si os sirve):
bucket_years = list(years_dob.sort_values().apply(lambda x: x//10*10).unique())
# Como los buckets los estoy haciendo por defecto, me coge el de la izq abierto y el de la derecha cerrado, por lo que tendríamos que tomar como mínimo 1 menos,
#y como máximo el máximo de todo (ya que ahora lo que estamos sacando como máximo es la década del máximo, ya que si el máximo es 2002, tenemos 2000 solo). Sin
#embargo, viendo los datos, comprobamos que el mínimo es 1977, por lo que la década 1970 ya lo incluye:
buckets_years = bucket_years + [years_dob.max()]
years = pd.cut(years_dob, buckets_years)

# Sacamos los buckets de los valores:
value_eur_buckets = pd.cut(df['value_eur'], [0, 100000, 1000000, max(df['value_eur'])])

# Sacamos la pivot_table:
df.pivot_table(["potential", "overall"], index = years, columns=value_eur_buckets, aggfunc={"potential": "mean", "overall": ["min", "max"]})

overall                                                     \
                     max                                                min   
value_eur    (0, 100000] (100000, 1000000] (1000000, 105500000] (0, 100000]   
dob                                                                           
(1970, 1980]          67                75                   83          57   
(1980, 1990]          68                76                   94          50   
(1990, 2000]          58                69                   92          48   
(2000, 2002]          56                65                   76          48   

                                                      potential  \
                                                           mean   
value_eur    (100000, 1000000] (1000000, 105500000] (0, 100000]   
dob                                                               
(1970, 1980]                65                   73   62.714286   
(1980, 1990]                58                   69   59.871622   
(1990, 2000]                52                   65   62.557491   
(2000, 2002]                52                   65   68.691781   

                                                     
                                                     
value_eur    (100000, 1000000] (1000000, 105500000]  
dob                                                  
(1970, 1980]         70.521739            77.333333  
(1980, 1990]         66.021084            74.594203  
(1990, 2000]         69.748005            77.509765  
(2000, 2002]         76.020349            84.677419